In [ ]:
# -*- coding: utf-8 -*-
"""Lat-Crawling Youtube Comment.ipynb



CRAWLING DATA COMMENTS
"""

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Masukkan API Key YouTube Data API v3
API_KEY = 'API KEY here'
VIDEO_ID = '0pYcAs1egFI'

# Bangun layanan API
youtube = build('youtube', 'v3', developerKey=API_KEY)

def get_video_comments(video_id, max_comments=10000):
    comments = []
    next_page_token = None

    while len(comments) < max_comments:
        try:
            # Panggil YouTube API untuk mendapatkan komentar
            request = youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                maxResults=1000,
                pageToken=next_page_token,
                textFormat='plainText'
            )
            response = request.execute()

            # Tambahkan komentar ke daftar
            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
                comments.append(comment)
                if len(comments) >= max_comments:
                    break

            # Dapatkan token halaman berikutnya
            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break
        except HttpError as e:
            print(f"An error occurred: {e}")
            break

    return comments

# Ambil komentar dari video
comments = get_video_comments(VIDEO_ID, max_comments=10000)
for i, comment in enumerate(comments, start=1):
    print(f"{i}. {comment}")

Output streaming akan dipotong hingga 5000 baris terakhir.
5989. Menyusul prindavan 😂
5990. WHY 🦅
5991. makin ke sini, IQ jongkok tetep banyak juga, kirain berkurang
5992. Jgn serius..ini cm lucu2an kok..
Biar pak Menteri ngga pusing saat ngojek lg sepi narik...menteri tukang ojek..
5993. UN ga ada, semangat belajar & berkompetisi ga ada 😂
5994. kalau anak sekarang lebih bodoh dari anak jaman dulu, apakah anak yang jaman dulu (sekarang sedang berkuasa/berumur 30an) saat ini sudah bermanfaat? saya pernah bicara sama anak-anak kurikulum merdeka, ilmu umum begini memang ga paham sih, tapi debat sama mereka seru karena mereka dari kecil diajarkan mikir, bukan diajarkan menghapal
5995. Semoga cepet Indonesia bubar 2030 😂
5996. Yg perlu diajarkan:
1. Adab
2. Baca tulis
3. Hitung
4. Pikir kritis
5. Tekno

Asal bisa baca dan tekno, pengetahuan umum bisa diperoleh dgn mudah.

Masalahnya adab dan pikir kritis kurang jd gak bisa menyaring info2.
5997. tapi si Nadiem masih kepedean
5998. Menitik b

In [ ]:
"""MERUBAH TEXT MENJADI DATAFRAME"""

import pandas as pd

# Membuat DataFrame dari comments dengan kolom 'Comment'
comments_df = pd.DataFrame(comments, columns=['Comment'])

# Menampilkan DataFrame
print(comments_df)

                                                Comment
0                        Bohong, gak mungkin sebogo itu
1     Kur. merdeka penuh kebebasan, kemampuan, keing...
2     NI SALAH MENTRI PENDIDIKAN  SAAT NI TIDAK BENA...
3             Ini sdm² bekal untuk suara jan etes nanti
4           Misi Yahudi sepuluh tahun terakhir berhasil
...                                                 ...
9555  Terlalu di tuntut menuntut ilmu eksakta dan il...
9556                                        Ambyarrrrrr
9557                                          Amburadul
9558  Salah gurunya ... gurunya hanya tahu.. scinker...
9559  Miriiiissss... dulu kami UUD saja sampai hafal...

[9560 rows x 1 columns]


In [ ]:
"""PEMBERSIHAN DATA TEXT"""

!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.9 MB/s eta 0:00:00


In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import re

# Pemasangan penghapus stop word  for bahasa indonesia Indonesian
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()


# Fungsi Preprocessing : lowercasing, removing punctuation, and stop words
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove stop words
    text = stopword.remove(text)
    return text


# Aplikasi preprocessing
comments_df['cleaned_comment'] = comments_df['Comment'].apply(preprocess_text)
comments_df[['Comment', 'cleaned_comment']].head()

,Comment,cleaned_comment
0,"Bohong, gak mungkin sebogo itu",bohong gak mungkin sebogo
1,"Kur. merdeka penuh kebebasan, kemampuan, keing...",kur merdeka penuh kebebasan kemampuan keingina...
2,NI SALAH MENTRI PENDIDIKAN SAAT NI TIDAK BENA...,ni salah mentri pendidikan ni benar benar mel...
3,Ini sdm² bekal untuk suara jan etes nanti,sdm² bekal suara jan etes
4,Misi Yahudi sepuluh tahun terakhir berhasil,misi yahudi sepuluh tahun terakhir berhasil


In [ ]:
"""RUBAH KATA SLANG MENJADI KATA BAKU"""

import nltk
from nltk.tokenize import word_tokenize

# Pastikan resource tokenization NLTK diunduh
nltk.download('punkt_tab')

#load data bahasa slang
slang_df = pd.read_csv('kamusalay.csv')  # Pastikan file berada di direktori yang benar
slang_words = dict(zip(slang_df['slang'], slang_df['baku']))

# Fungsi untuk menggantikan kata-kata slang dengan kata baku
def replace_slang(text):
    words = word_tokenize(str(text).lower())  # Pastikan text diubah ke string
    new_words = [slang_words.get(word, word) for word in words]  # Gunakan .get() untuk aman
    return ' '.join(new_words)

# Terapkan fungsi ke kolom 'cleaned_comment' untuk membuat kolom baru dengan kata baku
comments_df['Comment_Baku'] = comments_df['cleaned_comment'].apply(replace_slang)

# Lihat hasilnya
print(comments_df[['Comment', 'cleaned_comment', 'Comment_Baku']].head())

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


                                             Comment  \
0                     Bohong, gak mungkin sebogo itu   
1  Kur. merdeka penuh kebebasan, kemampuan, keing...   
2  NI SALAH MENTRI PENDIDIKAN  SAAT NI TIDAK BENA...   
3          Ini sdm² bekal untuk suara jan etes nanti   
4        Misi Yahudi sepuluh tahun terakhir berhasil   

                                     cleaned_comment  \
0                          bohong gak mungkin sebogo   
1  kur merdeka penuh kebebasan kemampuan keingina...   
2  ni salah mentri pendidikan  ni benar benar mel...   
3                          sdm² bekal suara jan etes   
4        misi yahudi sepuluh tahun terakhir berhasil   

                                        Comment_Baku  
0                        bohong tidak mungkin sebogo  
1  kur merdeka penuh kebebasan kemampuan keingina...  
2  ni salah mentri pendidikan ni benar benar mela...  
3                          sdm² bekal suara jan etes  
4        misi yahudi sepuluh tahun terakhir berhasi

In [ ]:
"""MELABELI DATA TEXT DENGAN LABEL SENTIMEN POSITIF, NEGATIF ATAU NETRAL"""

from google.colab import files


# Fungsi untuk membaca file txt dan mengonversinya ke kamus
def load_sentiment_words(file_path):
    sentiment_words = {}
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            # Lewati baris kosong atau baris yang tidak memiliki spasi
            if not line or ' ' not in line:
                continue
            word, score = line.split(maxsplit=1)  # Pisahkan berdasarkan spasi pertama
            sentiment_words[word.strip()] = int(score.strip())
    return sentiment_words

# Load kamus sentimen dari file yang baru diunggah
sentiment_words = load_sentiment_words('/content/sentimentword.txt')

In [ ]:
# Fungsi untuk menghitung skor sentimen
def sentiment_score(text):
    words = word_tokenize(text.lower())
    score = sum(sentiment_words.get(word, 0) for word in words)
    return score

# Fungsi untuk mengklasifikasikan sentimen berdasarkan skor
def classify_sentiment(score):
    if score > 0:
        return 'Positif'
    elif score < 0:
        return 'Negatif'
    else:
        return 'Netral'

# Terapkan fungsi scoring dan klasifikasi ke kolom 'Comment'
comments_df['Score'] = comments_df['Comment_Baku'].apply(sentiment_score)
comments_df['Sentiment'] = comments_df['Score'].apply(classify_sentiment)

print(comments_df)

                                                Comment  \
0                        Bohong, gak mungkin sebogo itu   
1     Kur. merdeka penuh kebebasan, kemampuan, keing...   
2     NI SALAH MENTRI PENDIDIKAN  SAAT NI TIDAK BENA...   
3             Ini sdm² bekal untuk suara jan etes nanti   
4           Misi Yahudi sepuluh tahun terakhir berhasil   
...                                                 ...   
9555  Terlalu di tuntut menuntut ilmu eksakta dan il...   
9556                                        Ambyarrrrrr   
9557                                          Amburadul   
9558  Salah gurunya ... gurunya hanya tahu.. scinker...   
9559  Miriiiissss... dulu kami UUD saja sampai hafal...   

                                        cleaned_comment  \
0                             bohong gak mungkin sebogo   
1     kur merdeka penuh kebebasan kemampuan keingina...   
2     ni salah mentri pendidikan  ni benar benar mel...   
3                             sdm² bekal suara jan etes

In [ ]:
"""EXPORT DATA KEDALAM FILE XLSX"""

comments_df.to_excel('sentimenpendidikan.xlsx', index=False)